# Day 26: Kolmogorov Complexity & Algorithmic Randomness

Day 26 of 30 Papers in 30 Days.

Today: **Kolmogorov Complexity ($C(x)$)** — the ultimate measure of information content. While Shannon entropy tells us about the source, Kolmogorov tells us about the object itself.

## What We'll Do

1. **The Problem**: why statistical entropy isn't enough
2. **The Solution**: Compression as a proxy for intelligence
3. **The Walkthrough**: Building a Huffman Tree bit-by-bit
4. **The conveyor belt**: Arithmetic Interval Shrinkage
5. **Similarity without Models**: Clusterting with NCD
6. **The Incompressibility Proof**: Why most noise stays noise

## The Core Insight

The complexity of a string is the length of the shortest program that produces it. Understanding is the process of finding that program.

In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import random
import string
from collections import Counter

# Import our implementations
from implementation import HuffmanCoder, ArithmeticCoder, ComplexityMetrics

h_coder = HuffmanCoder()
a_coder = ArithmeticCoder()

print("All imports successful.")

## 1. The Problem: Intelligence as Compression

Shannon Information theory (1948) deals with transmission over noisy channels. It assumes data comes from a known 'distribution'.

But what if we have a single, fixed object? Like a DNA sequence or a file? 

**AIT Insight:** The information in $x$ is the length of the shortest description of $x$. If you understand the physics of a system, you can write a short simulation that produces its history. If you don't, you just have a list of random facts.

## 2. The Walkthrough: Huffman Step-by-Step

Huffman coding is a greedy algorithm. Let's watch it build a tree for the word `"KOLMOGOROV"`.

In [ ]:
text = "KOLMOGOROV"
freqs = Counter(text)
print(f"Frequencies: {dict(freqs)}")

# Visualize the sorted frequencies (The foundation of the tree)
labels, values = zip(*sorted(freqs.items(), key=lambda x: x[1]))
plt.bar(labels, values, color='#6d597a')
plt.title("Character Frequencies in 'KOLMOGOROV'")
plt.ylabel("Count")
plt.show()

codes = h_coder.generate_codes(text)
print("\nGenerated Prefix Codes:")
for char in sorted(codes.keys()):
    print(f"  '{char}': {codes[char]}")

## 3. The Conveyor Belt: Arithmetic Range Updates

Instead of mapping chars to bits, Arithmetic coding maps the entire sequence to a number in $[0, 1)$. We treat character probabilities as sub-intervals on a 'conveyor belt' that we zoom into.

In [ ]:
def trace_arithmetic(text):
    low, high = 0.0, 1.0
    # Simple uniform-ish probability for demonstration
    chars = sorted(list(set(text)))
    p = 1.0 / len(chars)
    char_ranges = {c: (i*p, (i+1)*p) for i, c in enumerate(chars)}
    
    print(f"{'Char':5} | {'Low':10} | {'High':10} | {'Range'}")
    print("-" * 50)
    
    for char in text[:6]: # Just first 6 for visibility
        p_low, p_high = char_ranges[char]
        width = high - low
        high = low + width * p_high
        low = low + width * p_low
        print(f"{char:5} | {low:10.8f} | {high:10.8f} | {high-low:10.8f}")

trace_arithmetic("KOLMOGOROV")

## 4. The Complexity Spectrum

We compare strings of equal length but different structure. $C(x)$ should be low for constant strings and high for random ones.

In [ ]:
length = 500
constant = "a" * length
periodic = "abcde" * (length // 5)
random_str = "".join(random.choices(string.ascii_lowercase, k=length))

for name, s in [("Constant", constant), ("Periodic", periodic), ("Random", random_str)]:
    bits = h_coder.get_complexity(s)
    ratio = bits / (length * 8)
    print(f"{name:12} | Compressed Bits: {bits:5} | Ratio: {ratio:5.2f}")

## 5. Similarity Without Models (NCD)

NCD measures similarity by checking if $C(xy)$ is smaller than $C(x) + C(y)$. If strings share patterns, their joint complexity drops.

In [ ]:
math1 = "Prime numbers are only divisible by one and themselves."
math2 = "There is no largest prime number; the set is infinite."
code1 = "for i in range(10): print(i**2)"

comp = lambda x: h_coder.get_complexity(x)
print(f"NCD(Math 1, Math 2): {ComplexityMetrics.ncd(math1, math2, comp):.3f} (Lower = More Similar)")
print(f"NCD(Math 1, Code 1): {ComplexityMetrics.ncd(math1, code1, comp):.3f}")

## 6. Summary

1. **Intelligence = Compression**: The ability to find regularities in data and represent them as short programs.
2. **The Bottleneck**: $C(x)$ is the theoretical limit of how much we can compress an object.
3. **Randomness**: A string is random if its shortest description is as long as the string itself.

### Next Steps
- Solve the **Incompressibility Challenge** in `exercises/exercise_05_incompressibility.py`.
- Explore the **Paper Notes** for the deep mathematical proofs from the Moscow School.